In [ ]:
import ctypes
from ctypes import wintypes
import win32con
import win32api
import win32gui
import win32process

In [ ]:
import sys
import os
import traceback
import win32con as wcon
import win32api as wapi
import win32gui as wgui
import win32process as wproc
 
 
# Callback
def enum_windows_proc(wnd, param):
    pid = param.get("pid", None)
    data = param.get("data", None)
    if pid is None or wproc.GetWindowThreadProcessId(wnd)[1] == pid:
        text = wgui.GetWindowText(wnd)
        if text:
            style = wapi.GetWindowLong(wnd, wcon.GWL_STYLE)
            if style & wcon.WS_VISIBLE:
                if data is not None:
                    data.append((wnd, text))
                #else:
                    print("%08X - %s" % (wnd, text))
 
 
def enum_process_windows(pid=None):
    data = []
    param = {
        "pid": pid,
        "data": data,
    }
    wgui.EnumWindows(enum_windows_proc, param)
    return data
 
 
def _filter_processes(processes, search_name=None):
    if search_name is None:
        return processes
    filtered = []
    for pid, _ in processes:
        try:
            proc = wapi.OpenProcess(wcon.PROCESS_ALL_ACCESS, 0, pid)
        except:
#             print("Process {0:d} couldn't be opened: {1:}".format(pid, traceback.format_exc()))
            continue
        try:
            file_name = wproc.GetModuleFileNameEx(proc, None)
        except:
            #print("Error getting process name: {0:}".format(traceback.format_exc()))
            wapi.CloseHandle(proc)
            continue
        base_name = file_name.split(os.path.sep)[-1]
        if base_name.lower() == search_name.lower():
            filtered.append((pid, file_name))
        wapi.CloseHandle(proc)
    return tuple(filtered)
 
 
def enum_processes(process_name=None):
    procs = [(pid, None) for pid in wproc.EnumProcesses()]
    return _filter_processes(procs, search_name=process_name)


In [ ]:
def fo():
    search_name='cloudmusic.exe'

    processes=wproc.EnumProcesses()#get all the process id
    pids=[]
    for pid in processes:
        try:
            proc = wapi.OpenProcess(wcon.PROCESS_ALL_ACCESS, 0, pid)
        except:
    #         print("Process {0:d} couldn't be opened: {1:}".format(pid, traceback.format_exc()))
            continue
        try:
            file_name = wproc.GetModuleFileNameEx(proc, None)
        except:
    #         print("Error getting process name: {0:}".format(traceback.format_exc()))
            wapi.CloseHandle(proc)
            continue
        base_name = file_name.split(os.path.sep)[-1]
        if base_name.lower() == search_name.lower():
            print(pid,base_name)
            pids.append(pid)
        wapi.CloseHandle(proc)
    return pids


In [ ]:
# Callback
def enum_windows_proc(wnd, param):
    pid = param.get("pid", None)
    data = param.get("data", None)
    if pid is None or wproc.GetWindowThreadProcessId(wnd)[1] == pid:
        text = wgui.GetWindowText(wnd)
        if text:
            style = wapi.GetWindowLong(wnd, wcon.GWL_STYLE)
            if style & wcon.WS_VISIBLE:
                if data is not None:
                    data.append((wnd, text))
                #else:
                    print("%d - %s" % (wnd, text))

for pid in pids:
    data = []
    param = {
        "pid": pid,
        "data": data,
    }
    wgui.EnumWindows(enum_windows_proc, param)


for pid in pids:
    tmp=wproc.GetWindowThreadProcessId(pid)
    print(type(tmp))
    print(pid)
    print(tmp)
# for pid in processes:
#     data = []
#     param = {
#         "pid": pid,
#         "data": data,
#     }
#     wgui.EnumWindows(enum_windows_proc, param)

In [ ]:
def foo(wnd,param):
    pids_=param["pids"]
    
    if wproc.GetWindowThreadProcessId(wnd)[1] in pids_:
        text = wgui.GetWindowText(wnd)
        if '-' in text:
            data[wnd]=text
    
data={}
wgui.EnumWindows(foo,{"pids":pids,"data":data})
print(data)

In [ ]:
import os
import win32con as wcon
import win32api as wapi
import win32gui as wgui
import win32process as wproc
from time import sleep

class get_music_name_now:
    def foo(self,wnd,param):
        pids_=param["pids"]

        if wproc.GetWindowThreadProcessId(wnd)[1] in pids_:
            text = wgui.GetWindowText(wnd)
            if '-' in text:
                self.data.append(wnd)

    def get_pids(self):
        search_name=self.search_name

        processes=wproc.EnumProcesses()#get all the process id
        pids=[]
        for pid in processes:
            try:
                proc = wapi.OpenProcess(wcon.PROCESS_ALL_ACCESS, 0, pid)
            except:
                continue
            try:
                file_name = wproc.GetModuleFileNameEx(proc, None)
            except:
                wapi.CloseHandle(proc)
                continue
            base_name = file_name.split(os.path.sep)[-1]
            if base_name.lower() == search_name.lower():
                pids.append(pid)
            wapi.CloseHandle(proc)
        return pids

    def __init__(self,obs_path,search_name='cloudmusic.exe'):
        self.obs_path=obs_path
        self.search_name=search_name
        self.text=""
        self.data=[]#record the handle of required procession
        
        self.pids=self.get_pids()
        wgui.EnumWindows(self.foo,{"pids":self.pids,"data":self.data})
        while len(self.data)==0:
            print('搜索未成功，进行休眠')
            self.pids=self.get_pids()
            wgui.EnumWindows(self.foo,{"pids":self.pids,"data":self.data})
            sleep(10)#这是没有读取到'cloudmusic.exe'的数据时休息的时间，单位为秒
            
    def __run__(self):
        path=self.obs_path
        
        text = wgui.GetWindowText(self.data[0])
        if len(text)==0:
            self.__init__(self.obs_path)
            text=wgui.GetWindowText(self.data[0])
        
        if self.text!=text:
            f=open(path,'w')
            f.write("当前bgm为:"+text)
            print("进行了写入")
            f.close()
            self.text=text
        print("当前bgm为:",text)
        
        sleep(3)#这是每次读取以后间隔的时间，单位是秒
      
if __name__ == "__main__":
    A=get_music_name_now(r"C:\Users\k-on\Desktop\i\jupyter\obs.txt")
    #这里需要传入参数，是需要写入的文件的地址，注意windows上路径复制过来以后字符串前面要加r
    while True:
        A.__run__()